In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from underthesea import word_tokenize, pos_tag, sent_tokenize
import warnings
from gensim import corpora, models, similarities
import jieba
import re

import warnings

warnings.filterwarnings('ignore')

In [2]:
STOP_WORD_FILE = 'Files/vietnamese-stopwords.txt'

In [3]:
with open(STOP_WORD_FILE, 'r', encoding='utf-8') as file:
    stop_words = file.read()

stop_words = stop_words.split('\n')

In [4]:
data = pd.read_csv('Files/Product.csv', index_col=0)

In [5]:
data.head()

,product_id,product_name,description,product_rating,group,price
0,48102821,Tai nghe Bluetooth Inpods 12 - Cảm biến vân ta...,Dung lượng pin 300 Thời gian pin - Thời gian ...,4.0,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,77000
1,52333193,Tai nghe bluetooth không dây F9 True wireless ...,Dung lượng pin 2000mah Thời gian pin Thời gia...,4.5,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,132000
2,299461,Chuột Không Dây Logitech M331 Silent - Hàng Ch...,Thương hiệu Logitech Độ phân giải quang học 1...,4.8,Thiết Bị Số - Phụ Kiện Số/Phụ kiện máy tính và...,299000
3,57440329,Loa Bluetooth 5.0 Kiêm Đồng Hồ Báo Thức - [[ 2...,Thương hiệu Acome Xuất xứ thương hiệu Indones...,4.7,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,149000
4,38458616,Tai Nghe Bluetooth Apple AirPods Pro True Wire...,Thương hiệu Apple Xuất xứ thương hiệu Mỹ Kích...,4.8,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,5090000


In [6]:
data['product'] = data['product_name'].astype(str) + data['description'].astype(str)

In [7]:
data.head()

,product_id,product_name,description,product_rating,group,price,product
0,48102821,Tai nghe Bluetooth Inpods 12 - Cảm biến vân ta...,Dung lượng pin 300 Thời gian pin - Thời gian ...,4.0,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,77000,Tai nghe Bluetooth Inpods 12 - Cảm biến vân ta...
1,52333193,Tai nghe bluetooth không dây F9 True wireless ...,Dung lượng pin 2000mah Thời gian pin Thời gia...,4.5,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,132000,Tai nghe bluetooth không dây F9 True wireless ...
2,299461,Chuột Không Dây Logitech M331 Silent - Hàng Ch...,Thương hiệu Logitech Độ phân giải quang học 1...,4.8,Thiết Bị Số - Phụ Kiện Số/Phụ kiện máy tính và...,299000,Chuột Không Dây Logitech M331 Silent - Hàng Ch...
3,57440329,Loa Bluetooth 5.0 Kiêm Đồng Hồ Báo Thức - [[ 2...,Thương hiệu Acome Xuất xứ thương hiệu Indones...,4.7,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,149000,Loa Bluetooth 5.0 Kiêm Đồng Hồ Báo Thức - [[ 2...
4,38458616,Tai Nghe Bluetooth Apple AirPods Pro True Wire...,Thương hiệu Apple Xuất xứ thương hiệu Mỹ Kích...,4.8,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,5090000,Tai Nghe Bluetooth Apple AirPods Pro True Wire...


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4373 entries, 0 to 4403
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   product_id      4373 non-null   int64  
 1   product_name    4373 non-null   object 
 2   description     4370 non-null   object 
 3   product_rating  4373 non-null   float64
 4   group           4373 non-null   object 
 5   price           4373 non-null   int64  
 6   product         4373 non-null   object 
dtypes: float64(1), int64(2), object(4)
memory usage: 273.3+ KB


In [9]:
# word_tokenize
data['product_wt'] = data['product'].apply(lambda x: word_tokenize(x, format = 'text'))

In [10]:
data.head()

,product_id,product_name,description,product_rating,group,price,product,product_wt
0,48102821,Tai nghe Bluetooth Inpods 12 - Cảm biến vân ta...,Dung lượng pin 300 Thời gian pin - Thời gian ...,4.0,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,77000,Tai nghe Bluetooth Inpods 12 - Cảm biến vân ta...,Tai_nghe Bluetooth_Inpods 12 - Cảm_biến vân ta...
1,52333193,Tai nghe bluetooth không dây F9 True wireless ...,Dung lượng pin 2000mah Thời gian pin Thời gia...,4.5,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,132000,Tai nghe bluetooth không dây F9 True wireless ...,Tai_nghe bluetooth không dây F9_True wireless ...
2,299461,Chuột Không Dây Logitech M331 Silent - Hàng Ch...,Thương hiệu Logitech Độ phân giải quang học 1...,4.8,Thiết Bị Số - Phụ Kiện Số/Phụ kiện máy tính và...,299000,Chuột Không Dây Logitech M331 Silent - Hàng Ch...,Chuột Không Dây Logitech_M331_Silent - Hàng Ch...
3,57440329,Loa Bluetooth 5.0 Kiêm Đồng Hồ Báo Thức - [[ 2...,Thương hiệu Acome Xuất xứ thương hiệu Indones...,4.7,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,149000,Loa Bluetooth 5.0 Kiêm Đồng Hồ Báo Thức - [[ 2...,Loa Bluetooth_5.0_Kiêm Đồng_Hồ Báo_Thức - [ [ ...
4,38458616,Tai Nghe Bluetooth Apple AirPods Pro True Wire...,Thương hiệu Apple Xuất xứ thương hiệu Mỹ Kích...,4.8,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,5090000,Tai Nghe Bluetooth Apple AirPods Pro True Wire...,Tai_Nghe Bluetooth_Apple_AirPods_Pro_True_Wire...


In [11]:
data["Index"] = [i for i in range(data.shape[0])]

### Gensim

In [12]:
products_gem = [[text for text in x.split()] for x in data.product_wt]

In [13]:
len(products_gem)

4373

In [14]:
products_gem[:1]

[['Tai_nghe',
  'Bluetooth_Inpods',
  '12',
  '-',
  'Cảm_biến',
  'vân',
  'tay',
  ',',
  'chống',
  'nước',
  ',',
  'màu_sắc',
  'đa_dạng',
  '-',
  '5',
  'màu_sắc',
  'lựa_chọn',
  'Dung_lượng',
  'pin',
  '300',
  'Thời_gian',
  'pin',
  '-',
  'Thời_gian',
  'nghe',
  'nhạc',
  'liên_tục',
  'từ',
  '2.5',
  '-',
  '4',
  'h',
  '-',
  'Thời_gian',
  'sạc',
  'đầy',
  'chỉ',
  'khoảng',
  '60',
  'p',
  '-',
  'Thời_gian',
  'chờ',
  'lên',
  'tới',
  '140',
  'giờ',
  'Bluetooth',
  '5',
  'Thương_hiệu',
  'OEM',
  'Xuất_xứ',
  'thương_hiệu',
  'Trung_Quốc',
  'Độ',
  'nhạy_cảm_biến',
  'vân',
  'tay',
  'Model',
  'i12',
  'Loại',
  'Jack',
  'cắm',
  'USB_Cable',
  'Trọng_lượng',
  '300',
  'g',
  'Thời_gian',
  'sử_dụng',
  '-',
  'Thời_gian',
  'nghe',
  'nhạc',
  'liên_tục',
  'từ',
  '2.5',
  '-',
  '4',
  'h',
  'SKU',
  '4096608751631',
  'MÔ_TẢ',
  'SẢN_PHẨM',
  'INPOD_12',
  'là',
  'phiên_bản',
  'nâng_cấp',
  'mới',
  'nhất',
  ',',
  'tai_nghe',
  'Bluetooth_5.0',

In [15]:
# remove some special elements in texts
products_gem_re = [[re.sub('[0-9]+','', e) for e in text] for text in products_gem] # số
products_gem_re = [[re.sub(r'\W+', '', e) for e in text] for text in products_gem_re] # special character
products_gem_re = [[re.sub('^_+|_+$', '', e) for e in text] for text in products_gem_re] # remove leading and trailing underscore
products_gem_re = [[re.sub(r'\b\w\b', '', e) for e in text] for text in products_gem_re] # một letter
products_gem_re = [[t.lower() for t in text if not t in ['', ' ', 'g', 'ml', '_v', '_.', '._']] for text in  products_gem_re] # ký tự đặc biệt
products_gem_re = [[t for t in text if not t in stop_words] for text in products_gem_re] # stopword

In [16]:
products_gem_re[:1]

[['tai_nghe',
  'bluetooth_inpods',
  'cảm_biến',
  'vân',
  'chống',
  'màu_sắc',
  'đa_dạng',
  'màu_sắc',
  'lựa_chọn',
  'dung_lượng',
  'pin',
  'pin',
  'nhạc',
  'liên_tục',
  'sạc',
  'chờ',
  'bluetooth',
  'thương_hiệu',
  'oem',
  'xuất_xứ',
  'thương_hiệu',
  'trung_quốc',
  'độ',
  'nhạy_cảm_biến',
  'vân',
  'model',
  'jack',
  'cắm',
  'usb_cable',
  'trọng_lượng',
  'nhạc',
  'liên_tục',
  'sku',
  'mô_tả',
  'sản_phẩm',
  'inpod',
  'phiên_bản',
  'nâng_cấp',
  'tai_nghe',
  'bluetooth',
  'thiết_kế',
  'tỉ_lệ',
  'chuẩn',
  'tai',
  'airpod',
  'hãng',
  'lược_bỏ',
  'nút',
  'bấm',
  'thân',
  'tai',
  'thay',
  'nút',
  'cảm_ứng',
  'dễ_dàng',
  'thuận_tiện',
  'thao_tác',
  'nhạc',
  'dễ_dàng',
  'chạm',
  'bluetooth',
  'kết_nối',
  'vô_cùng',
  'ổn_định',
  'bluetooth',
  'kết_nối',
  'vô_cùng',
  'ổn_định',
  'tai',
  'kết_nối',
  'dock',
  'sạc',
  'chất',
  'âm',
  'thời_lượng',
  'pin',
  'cải_thiện',
  'tối_ưu',
  'dock',
  'sạc',
  'tiện_lợi',
  'sạc',
  '

In [17]:
# Obtain the number of features based on dictionary: Use corpora.Dictionary
dictionary = corpora.Dictionary(products_gem_re)

In [18]:
# List of features in dictionary
dictionary.token2id

{'airpod': 0,
 'apple': 1,
 'bao_gồm': 2,
 'bluetooth': 3,
 'bluetooth_inpods': 4,
 'bấm': 5,
 'chi_phí': 6,
 'chuẩn': 7,
 'chạm': 8,
 'chất': 9,
 'chống': 10,
 'chờ': 11,
 'cải_thiện': 12,
 'cảm_biến': 13,
 'cảm_ứng': 14,
 'cắm': 15,
 'cồng_kềnh': 16,
 'dock': 17,
 'dung_lượng': 18,
 'dễ_dàng': 19,
 'giao': 20,
 'giá': 21,
 'hiện_hành': 22,
 'huawei': 23,
 'hàng': 24,
 'hãng': 25,
 'inpod': 26,
 'jack': 27,
 'kết_nối': 28,
 'lenovo': 29,
 'liên_tục': 30,
 'luật': 31,
 'lược_bỏ': 32,
 'lựa_chọn': 33,
 'model': 34,
 'màu_sắc': 35,
 'mô_tả': 36,
 'nhạc': 37,
 'nhạy_cảm_biến': 38,
 'nâng_cấp': 39,
 'nút': 40,
 'oem': 41,
 'oppo': 42,
 'phiên_bản': 43,
 'phát_sinh': 44,
 'phí': 45,
 'phương_thức': 46,
 'phụ_phí': 47,
 'pin': 48,
 'samsung': 49,
 'sku': 50,
 'sạc': 51,
 'sản_phẩm': 52,
 'tablet': 53,
 'tai': 54,
 'tai_nghe': 55,
 'thao_tác': 56,
 'thay': 57,
 'thiết_bị': 58,
 'thiết_kế': 59,
 'thuận_tiện': 60,
 'thuế': 61,
 'thân': 62,
 'thương_hiệu': 63,
 'thời_lượng': 64,
 'tiki': 65,
 't

In [19]:
# Numbers of features (word) in dictionary
feature_cnt = len(dictionary.token2id)

In [20]:
feature_cnt

34513

In [21]:
# Obtain corpus based on dictionary (dense matrix)
corpus = [dictionary.doc2bow(text) for text in products_gem_re]

In [22]:
corpus[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 5),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 2),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 2),
 (18, 1),
 (19, 2),
 (20, 1),
 (21, 1),
 (22, 1),
 (23, 1),
 (24, 2),
 (25, 1),
 (26, 1),
 (27, 1),
 (28, 3),
 (29, 1),
 (30, 3),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 2),
 (36, 1),
 (37, 4),
 (38, 1),
 (39, 1),
 (40, 2),
 (41, 1),
 (42, 1),
 (43, 1),
 (44, 1),
 (45, 1),
 (46, 1),
 (47, 1),
 (48, 4),
 (49, 1),
 (50, 1),
 (51, 5),
 (52, 3),
 (53, 1),
 (54, 3),
 (55, 3),
 (56, 1),
 (57, 1),
 (58, 2),
 (59, 1),
 (60, 1),
 (61, 1),
 (62, 1),
 (63, 2),
 (64, 1),
 (65, 1),
 (66, 1),
 (67, 1),
 (68, 1),
 (69, 1),
 (70, 1),
 (71, 1),
 (72, 1),
 (73, 1),
 (74, 1),
 (75, 2),
 (76, 2),
 (77, 1),
 (78, 1),
 (79, 1),
 (80, 1),
 (81, 1),
 (82, 1),
 (83, 1),
 (84, 2)]

In [23]:
# Use TF-IDF model to process corpus, obtaining index
tfidf = models.TfidfModel(corpus)
# Tính toán sự tương tự trong ma trận thưa thớt
index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features = feature_cnt)

In [24]:
product_ID = 52333193

In [25]:
product = data[data.product_id == product_ID].head(1)
product

,product_id,product_name,description,product_rating,group,price,product,product_wt,Index
1,52333193,Tai nghe bluetooth không dây F9 True wireless ...,Dung lượng pin 2000mah Thời gian pin Thời gia...,4.5,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,132000,Tai nghe bluetooth không dây F9 True wireless ...,Tai_nghe bluetooth không dây F9_True wireless ...,1


In [26]:
# Sản phẩm đang xem
name_description_pre = product['product_wt'].to_string(index=False)

In [27]:
name_description_pre

'Tai_nghe bluetooth không dây F9_True wireless D...'

In [28]:
view_product = name_description_pre

In [29]:
def recommender(view_product, dictionary, tfidf, index):
    # Convert search words into Sparse Vectors
    view_product = view_product.lower().split()
    kw_vector = dictionary.doc2bow(view_product)
    # print("View product's vector:")
    # print(kw_vector)
    # Similarity calculation
    sim = index[tfidf[kw_vector]]
    
    list_id = []
    list_score = []
    for i in range(len(sim)):
        list_id.append(i)
        list_score.append(sim[i])
    
    df_result = pd.DataFrame({'id': list_id,
                              'Similarity-Score': list_score})
    
    # top 5 highest scores
    five_highest_score = df_result.sort_values(by='Similarity-Score', ascending=False).head(6)
    idToList = list(five_highest_score['id'])
    
    product_find = data[data.Index.isin(idToList)]
    results = product_find[['Index', 'product_name']]
    results = pd.concat([results, five_highest_score], axis=1).sort_values(by='Similarity-Score', ascending=False)
    return results[['Similarity-Score', 'product_name']].iloc[1: , :]

In [30]:
recommender(view_product, dictionary, tfidf, index)

,Similarity-Score,product_name
968,0.338703,Tai Nghe True Wireless Earbuds QCY T9 Bluetoot...
587,0.332110,Tai nghe Bluetooth Lanith – Tai Nghe Không Dây...
719,0.320228,Tai nghe Blutooth 5.0 kiêm dock sạc dự phòng- ...
23,0.319422,Tai Nghe Bluetooth True Wireless AMOI F9 5.0 C...
119,0.306458,Tai Nghe Bluetooth True Wireless AMOI F9 5.0 C...


- Mua Tai_nghe bluetooth không dây F9 sẽ được recommend mua Tai Nghe True Wireless Earbuds QCY T9, Tai nghe Bluetooth Lanith,...